## 1. 데이터 불러오기 및 SHAP 요소를 통한 가중치검증

In [ ]:
import os
import math
import httpx 
import json
import shap
import random
import folium
import requests
import colorsys
import pandas as pd
import numpy as np
import xgboost as xgb
from dotenv import load_dotenv

In [26]:
# 사용자 데이터셋 파일 경로 입력
df = pd.read_csv("../data/processed/accident_with_hotspot.csv", low_memory=False)
df["is_elderly_accident"] = (df["acdnt_age_1_code"] >= 65).astype(int)
df["is_intersection_area"] = df["road_stle_dc"].str.contains("교차로").astype(int)
# 사고 여부 컬럼
y = df["is_elderly_accident"]

# 위험 요소 컬럼들 (데이터에 맞게 수정하세요)
X = df[[
    "is_intersection_area", # 교차로여부
    "near_crosswalk",       # 횡단보도 여부
    "near_traffic_light",   # 신호등 여부
    "lanes",                # 차선 수
    "near_elderly_zone",    # 노인보호구역 여부
    "near_child_zone",      # 어린이보호구역 여부
    "non_senior_hotspot",   # 일반사고다발지여부
    "senior_hotspot"        # 고령운전자 사고 다발지 여부
]]


# 모델 학습
model = xgb.XGBClassifier(eval_metric="logloss")
model.fit(X, y)

# SHAP 계산
explainer = shap.Explainer(model)
shap_values = explainer(X)

# 평균 SHAP 값 계산
mean_shap = np.abs(shap_values.values).mean(axis=0)
feature_weights = dict(zip(X.columns, mean_shap))

# 정규화 (총합 1로)
total = sum(feature_weights.values())
normalized_weights = {k: round(v / total, 4) for k, v in feature_weights.items()}

# 출력
print("🎯 가중치(정규화):")
for k, v in normalized_weights.items():
    print(f"{k}: {v}")



🎯 가중치(정규화):
is_intersection_area: 0.09030000120401382
near_crosswalk: 0.09179999679327011
near_traffic_light: 0.07479999959468842
lanes: 0.08349999785423279
near_elderly_zone: 0.030799999833106995
near_child_zone: 0.042399998754262924
non_senior_hotspot: 0.2694999873638153
senior_hotspot: 0.31690001487731934


In [ ]:
# 필요함수정의
def get_coords_from_keyword(keyword: str, api_key: str):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": keyword}
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
        
    if data.get("documents"):
        first_match = data["documents"][0]
        x = float(first_match["x"])  # 경도
        y = float(first_match["y"])  # 위도
        return {"x": x, "y": y}
    else:
        raise ValueError(f"🔍 '{keyword}'에 대한 결과가 없습니다.")

def extract_coord_list(df,lat_col="위도",lng_col="경도", accident_count=False):
    if accident_count:
        return list(zip(df[lat_col], df[lng_col], df["accident_count"])) 
    return list(zip(df[lat_col], df[lng_col]))  

def haversine(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: m)
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)

    a = math.sin(d_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(d_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # 거리 (m)

# 거리 계산 (빠른 하버사인)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    d_phi = math.radians(lat2 - lat1)
    d_lambda = math.radians(lon2 - lon1)
    a = math.sin(d_phi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(d_lambda/2)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

def interpolate_points(lat1, lon1, lat2, lon2, interval=5):
    distance = haversine(lat1, lon1, lat2, lon2)
    steps = max(1, int(distance // interval))
    lat_points = [lat1 + (lat2 - lat1) * i / steps for i in range(1, steps)]
    lon_points = [lon1 + (lon2 - lon1) * i / steps for i in range(1, steps)]
    return list(zip(lat_points, lon_points))

def is_within_haversine(center, coord_list, radius_m):
    return [coord for coord in coord_list if haversine(center[0], center[1], coord[0], coord[1]) <= radius_m]

def is_circle_overlap(center1, radius1, center2, radius2=50):
    dist = haversine(center1[0], center1[1], center2[0], center2[1])
    return dist <= (radius1 + radius2)

def get_hotspot_score(count, ranges):
    for (start, end, score) in ranges:
        if start <= count <= end:
            return score
    return 0.0  # // Changed

def random_color():
    h = random.random()              # 색상(Hue)
    s = 0.9                          # 채도 높게
    v = 0.9                          # 명도 높게
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return '#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255))

def risk_color(score):
    if score <= 0.4:
        return "green"
    elif score <= 0.7:
        return "orange"
    else:
        return "red"

In [44]:
load_dotenv()
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")

origin_keyword = "다이소 신림점"
destination_keyword = "서울특별시 보라매병원"

origin = get_coords_from_keyword(origin_keyword, KAKAO_API_KEY)
destination = get_coords_from_keyword(destination_keyword, KAKAO_API_KEY)

priority = "RECOMMEND"

url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}",
    "Content-Type": "application/json"
}
data = {
    "origin": origin,
    "destination": destination,
    "priority": priority
}

async def fetch_route():
    async with httpx.AsyncClient() as client:
        params = {
            "origin": f"{origin['x']},{origin['y']}",
            "destination": f"{destination['x']},{destination['y']}",
            "priority": priority
        }

        response = await client.get(
            url, headers=headers, params=params
        )
        response.raise_for_status()
        return response.json()

route_data = await fetch_route()

roads = route_data["routes"][0]["sections"][0]["roads"]
guides = route_data["routes"][0]["sections"][0]["guides"]

# 파일 로드
protection_path = "../data/external/protection_zone_data.csv"
crosswalk_path = "../data/external/crosswalk_data.csv"
traffic_light_path = "../data/external/traffic_light_data.csv"
senior_hotspots = "../data/raw/hotspot_info_senior.csv"
non_senior_hotspots = "../data/raw/hotspot_info_non_senior.csv"
intersection_path = "../data/external/intersection_data.csv"

protection_df = pd.read_csv(protection_path)
crosswalk_df = pd.read_csv(crosswalk_path)
traffic_light_df = pd.read_csv(traffic_light_path)
senior_hotsopt_df = pd.read_csv(senior_hotspots)
non_senior_hotspot_df = pd.read_csv(non_senior_hotspots)
intersection_df = pd.read_csv(intersection_path)
intersection_df = intersection_df.dropna(subset=["위도", "경도"])

protection_coords = extract_coord_list(protection_df)  
crosswalk_coords = extract_coord_list(crosswalk_df)  
traffic_light_coords = extract_coord_list(traffic_light_df)  
intersection_coords = extract_coord_list(intersection_df)
senior_hotspot_coords = extract_coord_list(senior_hotsopt_df, lat_col="hotspot_center_lat", lng_col="hotspot_center_lng", accident_count=True)  
non_senior_hotspot_coords = extract_coord_list(non_senior_hotspot_df, lat_col="hotspot_center_lat", lng_col="hotspot_center_lng", accident_count = True)  


In [ ]:
points = []
for road in roads:
    v = road["vertexes"]
    for i in range(0, len(v) - 2, 2):
        lat1, lon1 = v[i+1], v[i]
        lat2, lon2 = v[i+3], v[i+2]
        points.append((lat1, lon1))  # 시작점
        mids = interpolate_points(lat1, lon1, lat2, lon2, interval=25)
        points.extend(mids)          # 중간점
    points.append((v[-1], v[-2]))

# 중심점 계산
mid = points[len(points)//2]

fmap = folium.Map(location=mid, zoom_start=17)

all_scores = []
all_radii = []
risk_levels = {"safe": 0, "caution": 0, "danger": 0}

# 1. 점-점 사이 원 그리고 색상 표시
for i in range(len(points) - 1):
    lat1, lon1 = points[i]
    lat2, lon2 = points[i + 1]
    
    center = ((lat1 + lat2) / 2, (lon1 + lon2) / 2)
    radius = haversine(lat1, lon1, lat2, lon2) / 2
    
    #  위험요소 탐지
    in_intersection = is_within_haversine(center, intersection_coords, radius)
    in_protection = is_within_haversine(center, protection_coords, radius)
    in_crosswalk = is_within_haversine(center, crosswalk_coords, radius)
    in_traffic = is_within_haversine(center, traffic_light_coords, radius)
    
    senior_score_ranges = [
        (5, 7, 0.4),
        (8, 10, 0.45),
        (11, float('inf'), 0.5)
    ]

    non_senior_score_ranges = [
        (10, 12, 0.2),
        (13, 15, 0.25),
        (16, float('inf'), 0.3)
    ]
    
    in_senior = [coord for coord in senior_hotspot_coords if is_circle_overlap(center, radius, coord)]
    if bool(in_senior):        
        senior_cnt = sum(coord[2] for coord in in_senior)
        senior_score = get_hotspot_score(senior_cnt, senior_score_ranges)
    else:
        senior_score = 0
        
    in_non_senior = [coord for coord in non_senior_hotspot_coords if is_circle_overlap(center, radius, coord)]
    if bool(in_non_senior):
        non_senior_cnt = sum(coord[2] for coord in in_non_senior)
        non_senior_score = get_hotspot_score(non_senior_cnt, non_senior_score_ranges)
    else: 
        in_non_senior = 0
        
    lanes = 2
    '''
    ⚠️로직추가필요, True False로 구성예정임 테스트할 땐 False 또는 True로 놓고 테스트하세요
    '''
    in_velocity = False
    in_volume = False
    
    '''
    테스트 (사고다발지제외 가중치 2배)
    score = ( 
        0.124 * bool(in_crosswalk) +
        0.2 * bool(in_traffic) +
        0.290 * bool(in_intersection) +
        0.034 * bool(in_protection) +
        0.114 * bool(in_velocity) + 
        0.074 * bool(in_volume) +
        0.164 * lanes
    )
    print(score)
    '''
    
    score = (
        0.062 * bool(in_crosswalk) +
        0.100 * bool(in_traffic) +
        0.145 * bool(in_intersection) +
        0.017 * bool(in_protection) +
        0.0681 * lanes +
        0.057 * bool(in_velocity) + 
        0.037 * bool(in_volume) +
        1.0 * senior_score +          
        1.0 * non_senior_score + 
        0.0681 * lanes
    )
    
    # score = (
    #     0.0889 * bool(in_crosswalk) +
    #     0.1000 * bool(in_traffic) +
    #     0.0822 * bool(in_intersection) +
    #     0.0672 * bool(in_protection) +
    #     1.0 * senior_score +          
    #     1.0 * non_senior_score + 
    #     0.0681 * lanes
    # )
    
    #  점수 저장
    all_scores.append(score)
    all_radii.append(radius)
        
    color = risk_color(score)
    
    if score < 0.4:
        risk_levels["safe"] += 1
    elif score < 0.7:
        risk_levels["caution"] += 1
    else:
        risk_levels["danger"] += 1

    # 위험 설명 추가
    tooltip_lines = [f"위험 점수: {score*100:.2f}점"]

    if in_senior:
        tooltip_lines.append("✅근처에 고령자 사고다발지 있음")
    if in_non_senior:
        tooltip_lines.append("✅일반 사고다발지와 겹침")
    if in_intersection:
        tooltip_lines.append("✅교차로 포함")
    if in_protection:
        tooltip_lines.append("✅보호구역 포함")
    if in_crosswalk:
        tooltip_lines.append("✅횡단보도 있음")
    if in_traffic:
        tooltip_lines.append("✅신호등 있음")
        
    tooltip_text = "<br>".join(tooltip_lines)

    # 원 그리기
    folium.Circle(
        location=center,
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.3,
        tooltip=tooltip_text
    ).add_to(fmap)

# 모든 좌표 점 찍기
for lat, lon in points:
    folium.CircleMarker(
        location=(lat, lon),
        radius=3,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.8
    ).add_to(fmap)
    
#  평균 위험 점수
mean_score = sum(all_scores) / len(all_scores)

#  거리 가중 평균 점수
weighted_score = sum(s * r for s, r in zip(all_scores, all_radii)) / sum(all_radii)

#  비율 계산
total = sum(risk_levels.values())
safe_pct = risk_levels["safe"] / total * 100
caution_pct = risk_levels["caution"] / total * 100
danger_pct = risk_levels["danger"] / total * 100

##  요약 문자열 구성
# summary = (
#     f"<b>📊 평균 위험 점수:</b> {mean_score:.2f}<br>"
#     f"<b>📏 거리 가중 위험 점수:</b> {weighted_score:.2f}<br>"
#     f"<b>🟢 안전 구간 비율:</b> {safe_pct:.0f}%<br>"
#     f"<b>🟡 주의 구간 비율:</b> {caution_pct:.0f}%<br>"
#     f"<b>🔴 위험 구간 비율:</b> {danger_pct:.0f}%"
# )

# folium.Marker(
#     location=mid,  # 또는 중심 근처 다른 지점
#     icon=folium.DivIcon(
#         html=f"""
#         <div style="
#             font-size: 13px;
#             background-color: white;
#             border: 1px solid gray;
#             padding: 8px;
#             border-radius: 6px;
#             box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
#             width: 180px;             /*  줄 길이 고정 */
#             white-space: normal;      /*  줄바꿈 허용 */
#             line-height: 1.4;         /*  행간 조정 */
#         ">
#             📊 평균 위험 점수: {mean_score*100:.2f}<br>
#             📏 거리 가중 위험 점수: {weighted_score*100:.2f}<br>
#             🟢 안전 구간 비율: {safe_pct:.0f}%<br>
#             🟡 주의 구간 비율: {caution_pct:.0f}%<br>
#             🔴 위험 구간 비율: {danger_pct:.0f}%
#         </div>
#         """
#     )
# ).add_to(fmap)

# print(summary)
    
for _, row in senior_hotsopt_df.iterrows():
    lat, lon = row["hotspot_center_lat"], row["hotspot_center_lng"]

    # 1. 실제 반경 100m 원 그리기
    folium.Circle(
        location=(lat, lon),
        radius=50,  #  미터 단위
        color="pink",
        fill=True,
        fill_color="pink",
        fill_opacity=0.5
    ).add_to(fmap)

    # 2. 중심점 점 마커도 추가 (원 안에 표시)
    folium.CircleMarker(
        location=(lat, lon),
        radius=4,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.9,
    ).add_to(fmap)  

# for _, row in non_senior_hotspot_df.iterrows():
#     lat, lon = row["hotspot_center_lat"], row["hotspot_center_lng"]

#     # 1. 실제 반경 100m 원 그리기
#     folium.Circle(
#         location=(lat, lon),
#         radius=50,  #  미터 단위
#         color="skyblue",
#         fill=True,
#         fill_color="skyblue",
#         fill_opacity=0.5
#     ).add_to(fmap)

#     # 2. 중심점 점 마커도 추가 (원 안에 표시)
#     folium.CircleMarker(
#         location=(lat, lon),
#         radius=4,
#         color="skyblue",
#         fill=True,
#         fill_color="skyblue",
#         fill_opacity=0.9,
#     ).add_to(fmap)  
    
fmap